In [93]:
import numpy as np
import pandas as pd
from numpy import log2 as log
eps = np.finfo(float).eps
t_data = pd.read_csv("./dataset/train.csv")

In [94]:
t_data = t_data.drop(['Name', 'Ticket', 'Cabin', 'PassengerId'], axis = 1)
t_data = t_data.dropna()

In [95]:
t_data['Age'] = t_data['Age'].interpolate() #fixing missing values 

In [96]:
#Entropy of the entire dataset
entropy = 0
values = t_data['Survived'].unique()
for value in values:
    p = t_data['Survived'].value_counts()[value]/len(t_data.Survived)
    entropy = entropy + (-(p*log(p)))
print(entropy)

0.9735190023846809


In [97]:
def attribute_entropy(data, col):
    #print("Calculating att-ent")
    y = data['Survived'].unique()
    x = data[col].unique()
    entropy = 0
    for i in x:
        e = 0
        for j in y:
            num = len(data[col][data[col] == i][data['Survived'] == j])
            den = len(data[col][data[col] == i])
            p = num/(den+eps)
            e = e + (-(p*log(p+eps)))
        p2 = den/len(data)
        entropy = entropy + (-(p2*e))
    return abs(entropy)

In [98]:
print(attribute_entropy(t_data, 'Sex'))

0.7594106895489574


In [99]:
def find_max(d):
    #print("finding max")
    data = d.drop(['Survived'], axis = 1)
    att = []
    info_gain = []
    #print(data.keys())
    for key in data.keys()[:-1]:
        info_gain.append(entropy-attribute_entropy(d, key))
    #print(IG)
    return data.keys()[np.argmax(info_gain)]
find_max(t_data)

'Fare'

In [100]:
def get_subtable(df, node,value):
    #print('making a subtable')
    return df[df[node] == value].reset_index(drop=True)


def buildTree(data): 
    
    node = find_max(data)
    
    vals = np.unique(data[node])
    #print(vals)
    tree = {}
    tree[node] = {}
    for val in vals:
        subtable = get_subtable(data, node, val)
        #print((subtable.head()))
        
        if(len(set(subtable['Survived'])) == 1):
            tree[node][val] = subtable['Survived'].iloc[0]
        else:
            tree[node][val] = buildTree(subtable)
        print(tree)
    return tree

tree = buildTree(t_data)
print(tree)
  

{'Age': {19.0: 0}}
{'Age': {19.0: 0, 25.0: 1}}
{'Age': {19.0: 0, 25.0: 1, 36.0: 0}}
{'Age': {19.0: 0, 25.0: 1, 36.0: 0, 38.0: 0}}
{'Age': {19.0: 0, 25.0: 1, 36.0: 0, 38.0: 0, 39.0: 0}}
{'Age': {19.0: 0, 25.0: 1, 36.0: 0, 38.0: 0, 39.0: 0, 40.0: 0}}
{'Age': {19.0: 0, 25.0: 1, 36.0: 0, 38.0: 0, 39.0: 0, 40.0: 0, 49.0: 0}}
{'Fare': {0.0: {'Age': {19.0: 0, 25.0: 1, 36.0: 0, 38.0: 0, 39.0: 0, 40.0: 0, 49.0: 0}}}}
{'Fare': {0.0: {'Age': {19.0: 0, 25.0: 1, 36.0: 0, 38.0: 0, 39.0: 0, 40.0: 0, 49.0: 0}}, 4.0125: 0}}
{'Fare': {0.0: {'Age': {19.0: 0, 25.0: 1, 36.0: 0, 38.0: 0, 39.0: 0, 40.0: 0, 49.0: 0}}, 4.0125: 0, 5.0: 0}}
{'Fare': {0.0: {'Age': {19.0: 0, 25.0: 1, 36.0: 0, 38.0: 0, 39.0: 0, 40.0: 0, 49.0: 0}}, 4.0125: 0, 5.0: 0, 6.2375: 0}}
{'Fare': {0.0: {'Age': {19.0: 0, 25.0: 1, 36.0: 0, 38.0: 0, 39.0: 0, 40.0: 0, 49.0: 0}}, 4.0125: 0, 5.0: 0, 6.2375: 0, 6.4375: 0}}
{'Fare': {0.0: {'Age': {19.0: 0, 25.0: 1, 36.0: 0, 38.0: 0, 39.0: 0, 40.0: 0, 49.0: 0}}, 4.0125: 0, 5.0: 0, 6.2375: 0, 6.4375: 

RecursionError: maximum recursion depth exceeded while calling a Python object